#### Assignment 04
Für die Realisierung der Facettensuche von Medien muss das aktuelle Medium-Mapping von HTWplus angepasst werden.
Dazu benötigt es neue Felder, mit den Werten, die hinterher aggregiert werden sollen.

#### Aufgabe
Erweitere das **Medium-Mapping** so, damit folgende Aggregationen ausgeführt werden können:
- 1) Eine Gruppierung nach Eigentümern (Vor- und Nachname) **(2 Punkte)** ✓
    - Hinzugeüft wurde "ownername" zu "medium" vom type "keyword"
    - Im Invertierten Index steht so der Komplette Name als 1 Wort.  


- 2) Eine Gruppierung nach Speicherort (Root-Folder). **(2 Punkte)** ✓
    - Dem Index wurde ein filepath_analyzer hinzugefügt
    - Spaltet `path/to/file/` demnach in ["path/", "path/to/", "path/to/file/"]
    - Dem medium mapping wurde ein `filepath` hinzugefügt vom type text
    - Wird beim indexieren + suchen mit dem custom filepath_analyzer bearbeitet  


- 3) Eine Gruppierung nach Upload-Datum. Flexibel nach Tag, Monat oder Jahr. **(4 Punkte)** ✓
    - Möglichkeit 1: Man spaltet das auf in day(short), month(short) und year(short)
    - Möglichkeit 2: Man benutzt das date objekt. allerdings konnte ich hier nicht
      herausfinden ob das vernünftig aufgespalten werden kann, deshalb:
    - Ich hab mich für 1 entschieden :D
    
    
- 4) Eine Gruppierung nach Dateiendungen. **(2 Punkte)** ✓

#### Abgabe
 - Als Abgabe zählt das Vorzeigen oder Zuschicken des erzeugten Medium- Mappings.

In [5]:
# Delete existing Index
curl -XDELETE 'localhost:9200/htwplus?pretty'

{
  "acknowledged" : true
}


#### 1. Eine Gruppierung nach Eigentümern (Vor- und Nachname)

Wir wollen später Aggregieren können:

    curl -XPOST 'localhost:9200/htwplus/medium/_search?size=0&pretty' -H 'Content-Type: application/json' -d'
    {
      "aggs": {
        "by_owner": {
          "filter": { "keyword": {"ownername": "Iven Zimmer" } }
        }
      }
    }'

In [6]:
# Prepare Index JSON
INDEX='
{
  "settings": {
    "analysis": {
      "analyzer": {
        "edgengram_analyzer": {
          "tokenizer": "htw_edge_ngram"
        },
        "filepath_analyzer" : {
            "tokenizer" : "path_hierarchy"
        }
      },
      "tokenizer": {
        "htw_edge_ngram": {
          "type": "edge_ngram",
          "min_gram": 2,
          "max_gram": 8,
          "token_chars": [
            "letter",
            "digit"
          ]
        }
      }
    }
  },
  "mappings": {
    "medium": {
      "properties": {
        "owner": {
          "type": "short"
        },
        "ownername": {
            "type": "keyword"
        },
        "viewable": {
          "type": "long"
        },
        "public": {
          "type": "boolean"
        },
        "filepath": {
          "type": "text",
          "analyzer": "filepath_analyzer",
          "search_analyzer": "filepath_analyzer"
        },
        "filename": {
          "type": "text",
          "analyzer": "edgengram_analyzer",
          "search_analyzer": "whitespace"
        },
        "filetype": {
            "type": "text"
        },
        "upload_day": {
            "type": "short"
        },
        "upload_month": {
            "type": "short"
        },
        "upload_year": {
            "type": "short"
        }
      }
    },
    "users": {
      "properties": {
        "studycourse": {
          "index": "not_analyzed",
          "type": "text"
        },
        "role": {
          "index": "not_analyzed",
          "type": "text"
        },
        "public": {
          "type": "boolean"
        },
        "initial": {
          "type": "keyword"
        },
        "degree": {
          "index": "not_analyzed",
          "type": "text"
        },
        "firstname": {
          "search_analyzer": "whitespace",
          "analyzer": "edgengram_analyzer",
          "type": "text"
        },
        "lastname": {
          "search_analyzer": "whitespace",
          "analyzer": "edgengram_analyzer",
          "type": "text"
        },
        "semester": {
          "index": "not_analyzed",
          "type": "text"
        },
        "avatar": {
          "type": "text"
        },
        "friends": {
          "type": "long"
        }
      }
    }
  }
}'

In [7]:
# => Medium
# name => firstname, lastname
# The string field is unsupported for indexes created in 5.x in favor of the text and keyword fields.
# change all 'string' => 'text'

# Since 2.4 the Edge-N-Gram has changed so we create a new custom analyzer for Version 5
# This needs to happen at the same time we create the mapping for our type(s)
# Otherwise ES does not know about the analyzer ( analyzer not found exception )

# After that we Setup The Mapping for Our Users which now include 'firstname' + 'lastname'
# Instead of 'name'
curl -XPUT 'localhost:9200/htwplus?pretty' -H 'Content-Type: application/json' -d "${INDEX}"

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


In [8]:
# Test The analyzer for User (Not really necessary, but I wanted to test that.)
curl -XPOST 'localhost:9200/htwplus/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "analyzer": "edgengram_analyzer",
  "text": ["Iven Zimmer"]
}'

{
  "tokens" : [
    {
      "token" : "Iv",
      "start_offset" : 0,
      "end_offset" : 2,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "Ive",
      "start_offset" : 0,
      "end_offset" : 3,
      "type" : "word",
      "position" : 1
    },
    {
      "token" : "Iven",
      "start_offset" : 0,
      "end_offset" : 4,
      "type" : "word",
      "position" : 2
    },
    {
      "token" : "Zi",
      "start_offset" : 5,
      "end_offset" : 7,
      "type" : "word",
      "position" : 3
    },
    {
      "token" : "Zim",
      "start_offset" : 5,
      "end_offset" : 8,
      "type" : "word",
      "position" : 4
    },
    {
      "token" : "Zimm",
      "start_offset" : 5,
      "end_offset" : 9,
      "type" : "word",
      "position" : 5
    },
    {
      "token" : "Zimme",
      "start_offset" : 5,
      "end_offset" : 10,
      "type" : "word",
      "position" : 6
    },
    {
      "token" : "Zimmer",
      "start_offset" : 5,
      "end_o

In [9]:
curl -XPOST 'localhost:9200/htwplus/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "analyzer": "filepath_analyzer",
  "text": ["/Users/zimmeri/elastic17/Klausur.pdf"]
}'

{
  "tokens" : [
    {
      "token" : "/Users",
      "start_offset" : 0,
      "end_offset" : 6,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "/Users/zimmeri",
      "start_offset" : 0,
      "end_offset" : 14,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "/Users/zimmeri/elastic17",
      "start_offset" : 0,
      "end_offset" : 24,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "/Users/zimmeri/elastic17/Klausur.pdf",
      "start_offset" : 0,
      "end_offset" : 36,
      "type" : "word",
      "position" : 0
    }
  ]
}
